## Case Study #3 - Foodie-Fi

#### Problem Statement
Subscription based businesses are super popular and Danny realised that there was a large gap in the market - he wanted to create a new streaming service that only had food related content - something like Netflix but with only cooking shows!

Danny finds a few smart friends to launch his new startup Foodie-Fi in 2020 and started selling monthly and annual subscriptions, giving their customers unlimited on-demand access to exclusive food videos from around the world!

Danny created Foodie-Fi with a data driven mindset and wanted to ensure all future investment decisions and new features were decided using data. This case study focuses on using subscription style digital data to answer important business questions.

Danny has shared the data design for Foodie-Fi and also short descriptions on each of the database tables - our case study focuses on only 2 tables but there will be a challenge to create a new table for the Foodie-Fi team.

#### Entity Relationship Diagram

![week3.png](week3.png)

Import modules

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as sql
pd.set_option('display.max_columns', None)

Initialize SQL

In [2]:
conn = sql.connect("week3.db")
cursor = conn.cursor() 
if os.stat("week3.db").st_size == 0:
    with open('week3-sql.txt','r') as file:
        script = file.read()
        script = script.replace('\n', ' ')
    cursor.executescript(script)

Verify tables

In [3]:
query = """SELECT name FROM sqlite_master WHERE type='table';"""
cursor.execute(query)
tables = [table[0] for table in cursor.fetchall()]
tables
print(f'The tables in the database are: {', '.join(tables)}')

The tables in the database are: plans, subscriptions


Fetch table information

In [4]:
for table in tables:
    print("=================================")
    print(f'Table [{table}]')
    df = pd.read_sql_query(f'SELECT * FROM {table}', conn)
    print(f'Dimensions: {df.shape[0]} rows x {df.shape[1]} columns\n')
    print(df.head())
    info_df = pd.DataFrame.from_dict({'Datatypes':df.dtypes, 'NULL count':df.isna().sum()})
    print()
    print(info_df)
    print()

Table [plans]
Dimensions: 5 rows x 3 columns

   plan_id      plan_name  price
0        0          trial    0.0
1        1  basic monthly    9.9
2        2    pro monthly   19.9
3        3     pro annual  199.0
4        4          churn    NaN

          Datatypes  NULL count
plan_id       int64           0
plan_name    object           0
price       float64           1

Table [subscriptions]
Dimensions: 2650 rows x 3 columns

   customer_id  plan_id  start_date
0            1        0  2020-08-01
1            1        1  2020-08-08
2            2        0  2020-09-20
3            2        3  2020-09-27
4            3        0  2020-01-13

            Datatypes  NULL count
customer_id     int64           0
plan_id         int64           0
start_date     object           0



In [5]:
def query(stmt: str):
    """Executes a given SQL statement and returns a Pandas DataFrame given the results.
    
    Parameters
    ----------
    stmt: str
        The SQL statement to be executed
    """
    global conn
    result = pd.read_sql_query(stmt, conn)
    return result

## Case Study Questions

This case study is split into an initial data understanding question before diving straight into data analysis questions before finishing with 1 single extension challenge.

**A. Customer Journey**

Q1: Based off the 8 sample customers provided in the sample from the subscriptions table, write a brief description about each customer’s onboarding journey.

Try to keep it as short as possible - you may also want to run some sort of join to make your explanations a bit easier!

**B. Data Analysis Questions**

Q2: How many customers has Foodie-Fi ever had?

Q3: What is the monthly distribution of trial plan start_date values for our dataset - use the start of the month as the group by value

Q4: What is the customer count and percentage of customers who have churned rounded to 1 decimal place?

Q5: How many customers have churned straight after their initial free trial - what percentage is this rounded to the nearest whole number?

Q6: What is the number and percentage of customer plans after their initial free trial?

Q7: What is the customer count and percentage breakdown of all 5 plan_name values at 2020-12-31?

Q8: How many customers have upgraded to an annual plan in 2020?

Q9: How many days on average does it take for a customer to an annual plan from the day they join Foodie-Fi?

Q10: Can you further breakdown this average value into 30 day periods (i.e. 0-30 days, 31-60 days etc)

Q11: How many customers downgraded from a pro monthly to a basic monthly plan in 2020?

**C. Challenge Payment Question**

Q12: The Foodie-Fi team wants you to create a new payments table for the year 2020 that includes amounts paid by each customer in the subscriptions table with the following requirements:

- monthly payments always occur on the same day of month as the original start_date of any monthly paid plan
- upgrades from basic to monthly or pro plans are reduced by the current paid amount in that month and start immediately
- upgrades from pro monthly to pro annual are paid at the end of the current billing period and also starts at the end of the month period
- once a customer churns they will no longer make payments

**D. Additional Questions (Open-Ended)**

Q13: How would you calculate the rate of growth for Foodie-Fi?

Q14: What key metrics would you recommend Foodie-Fi management to track over time to assess performance of their overall business?

Q15: What are some key customer journeys or experiences that you would analyse further to improve customer retention?

Q16: If the Foodie-Fi team were to create an exit survey shown to customers who wish to cancel their subscription, what questions would you include in the survey?

Q17: What business levers could the Foodie-Fi team use to reduce the customer churn rate? How would you validate the effectiveness of your ideas?
Conclusion